In [1]:
from gold_docker import GOLDDocker
from ccdc.io import MoleculeReader, MoleculeWriter

In [2]:
protein_path = '/home/benoit/DUD-E/all/cp3a4/receptor.pdb'
native_ligand_path = '/home/benoit/DUD-E/all/cp3a4/crystal_ligand.mol2'
experiment_id = 'test_cp3a4_docking'
output_dir = '.'
binding_site_distance = 12

In [3]:
gold_docker = GOLDDocker(protein_path, native_ligand_path, experiment_id, output_dir,
                   binding_site_distance=binding_site_distance)

In [4]:
actives_path = '../DUD-E/all/cp3a4/actives_final.mol2.gz'
active_mols_reader = MoleculeReader(actives_path)
active_mols = [mol for mol in active_mols_reader]

In [5]:
active_mol = active_mols[0]

In [6]:
decoys_path = '../DUD-E/all/cp3a4/decoys_final.mol2.gz'
decoy_mols_reader = MoleculeReader(decoys_path)
decoy_mols = [mol for mol in decoy_mols_reader]
decoy_mol = decoy_mols[0]

In [7]:
decoy_mol

In [8]:
from ccdc_rdkit_connector import CcdcRdkitConnector
rdkit_mol = CcdcRdkitConnector().ccdc_mol_to_rdkit_mol(decoy_mol)

In [9]:
from molconfviewer import MolConfViewer
MolConfViewer().view(rdkit_mol)

interactive(children=(BoundedIntText(value=0, description='Conformer ID:', max=0), Output()), _dom_classes=('w…

BoundedIntText(value=0, description='Conformer ID:', max=0)

In [11]:
mol_id = 'decoy_0'
n_poses = 1
rigid = True
gold_docker.dock_molecule(decoy_mol, mol_id, n_poses, rigid)

Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/decoy_0/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/decoy_0/api_gold.conf"



In [68]:
from ccdc.docking import Docker
from ccdc.io import Entry
docker = Docker()
ligand_preparation = Docker.LigandPreparation()

settings = docker.settings
settings.fitness_function = 'plp'
settings.add_protein_file(protein_path)
settings.reference_ligand_file = native_ligand_path
protein = settings.proteins[0]
native_ligand = MoleculeReader(native_ligand_path)[0]
bs = settings.BindingSiteFromLigand(protein=protein, ligand=native_ligand)
settings.binding_site = bs

settings.write_options = ['NO_FIT_PTS_FILES', 
                                       'NO_LOG_FILES',
                                       'NO_RNK_FILES']

settings.clear_ligand_files()
ligand_filename = 'test_ligand.mol2'
ligand = ligand_preparation.prepare(Entry.from_molecule(active_mol))
# with MoleculeWriter(ligand_filename) as writer :
#     writer.write(active_mol)
mol2_string = ligand.to_string(format='mol2')
with open(ligand_filename, 'w') as writer :
    writer.write(mol2_string)
settings.add_ligand_file(ligand_filename, 1)
settings.fix_ligand_rotatable_bonds = 'all'

settings.output_directory = '/home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/active_0'
settings.output_file = '/home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/active_0/docked_ligands.mol2'
docker.dock()

Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/api_gold.conf"

Fatal error: 
do_ullman: query size larger than target



In [61]:
mol_id = 'active_0'
n_poses = 1
rigid = True
gold_docker.dock_molecule(active_mol, mol_id, n_poses, rigid)

Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/active_0/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/test_cp3a4_docking/active_0/api_gold.conf"

Fatal error: 
do_ullman: query size larger than target



FailedGOLDDockingException: GOLD did not return 0, docking failed

In [66]:
gold_docker.settings